In [1]:
%matplotlib inline
from IPython.display import clear_output

In [2]:
import numpy as np
import pylab as plt

from xgboost import XGBClassifier
from utils.data import load_data
from utils.pass_quality import pass_quality
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, GRU, SimpleRNN

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 4007)


In [3]:
def nn_model(X_train):
    model = Sequential()
    #model.add(InputLayer(input_shape=))
    model.add(Dense(12, activation='sigmoid', input_dim=X_train.shape[1]))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                  loss='MSE')
                #  metrics=['accuracy'])
    return model

# default classifiers like XGBoost, Logistic Regression

In [5]:
errors = []

for iteration in range(10):
    data = load_data()
    X, y = data[['cor', 'loop', 'shield']].values, data['y'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    #xgb = XGBClassifier()
    lr = LogisticRegression()
    #nn = nn_model(X_train)

    #xgb.fit(X_train, y_train)
    lr.fit(X_train, y_train)
    #nn.fit(X_train, y_train, nb_epoch=4, verbose=False)

    #y_pred_xgb = xgb.predict(X_test)
    y_pred_lr = lr.predict(X_test)
    #y_pred_nn = (nn.predict(X_test) > 0.5).astype(int).flatten()
    errors.append(pass_quality(y_test, y_pred_lr))

In [14]:
print (np.mean(np.matrix(errors).T[0]) + np.max(np.matrix(errors).T[0])) / 2

print (np.mean(np.matrix(errors).T[1]) + np.max(np.matrix(errors).T[1])) / 2

print (np.mean(np.matrix(errors).T[2]) + np.max(np.matrix(errors).T[2])) / 2

45.3934665197
2604.85


IndexError: index 2 is out of bounds for axis 0 with size 2

# data augmentation

In [7]:
def dim_extend(X, y, n_prev = 3):
    """"""
    X_ext, y_ext = [], []
    
    for i in range(len(X) - n_prev + 1):
        X_ext.append(np.concatenate([X[i+j] for j in range(n_prev)]))
        y_ext.append(y[i + n_prev - 1])
        #docY_lab.append(y_lab[i + n_prev -1])
    X_ext = np.array(X_ext)
    y_ext = np.array(y_ext)
    #alsY_lab = np.array(docY_lab)
    return X_ext, y_ext#, alsY_lab

# xgb model

In [10]:
errors_xgb = []

for n_prev in range(3, 25):
    print "computing for ", n_prev
    clear_output(wait=True)
    X_e, y_e = dim_extend(X, y, n_prev)
    X_train_e, X_test_e = X_e[:int(len(X_e) * 0.75)], X_e[int(len(X_e) * 0.75):]
    y_train_e, y_test_e = y_e[:int(len(y_e) * 0.75)], y_e[int(len(y_e) * 0.75):]
    xgb = XGBClassifier()
    xgb.fit(X_train_e, y_train_e)
    y_pred_e = xgb.predict(X_test_e)
    errors_xgb.append(pass_quality(y_test_e, y_pred_e))

computing for  24


In [11]:
print (np.max(np.matrix(errors_xgb).T[0]))

print (np.max(np.matrix(errors_xgb).T[1]))
#print np.max(np.matrix(errors_nn).T[1])

print (np.max(np.matrix(errors_xgb).T[2]))
#print np.max(np.matrix(errors_nn).T[2])

80.0980392157
1655.0
611.0


# logreg model

In [13]:
errors_lr = []

for n_prev in range(3, 30):
    print "computing for ", n_prev
    clear_output(wait=True)
    X_e, y_e = dim_extend(X, y, n_prev)
    X_train_e, X_test_e = X_e[:int(len(X_e) * 0.75)], X_e[int(len(X_e) * 0.75):]
    y_train_e, y_test_e = y_e[:int(len(y_e) * 0.75)], y_e[int(len(y_e) * 0.75):]
    lr = LogisticRegression()
    lr.fit(X_train_e, y_train_e)
    y_pred_e = lr.predict(X_test_e)
    errors_lr.append(pass_quality(y_test_e, y_pred_e))

computing for  29


In [15]:
print (np.max(np.matrix(errors_lr).T[0]))

print (np.max(np.matrix(errors_lr).T[1]))
#print np.max(np.matrix(errors_nn).T[1])

print (np.max(np.matrix(errors_lr).T[2]))
#print np.max(np.matrix(errors_nn).T[2])

76.51444548
1642.0


IndexError: index 2 is out of bounds for axis 0 with size 2

# nn model:

In [17]:
errors_nn = []

for n_prev in range(20, 25):
    print "computing for ", n_prev
    clear_output(wait=True)
    X_e, y_e = dim_extend(X, y, n_prev)
    X_train_e, X_test_e = X_e[:int(len(X_e) * 0.75)], X_e[int(len(X_e) * 0.75):]
    y_train_e, y_test_e = y_e[:int(len(y_e) * 0.75)], y_e[int(len(y_e) * 0.75):]
    nn = nn_model(X_train_e)
    nn.fit(X_train_e, y_train_e, nb_epoch=int(n_prev / 6.) + 6, verbose=False)
    y_pred_e = nn.predict(X_test_e)
    errors_nn.append(pass_quality(y_test_e, (nn.predict(X_test_e) > 0.5).astype(int).flatten()))

computing for  24


In [18]:
print (np.mean(np.matrix(errors_nn).T[0]))

print (np.mean(np.matrix(errors_nn).T[1]))
#print np.max(np.matrix(errors_nn).T[1])

print (np.mean(np.matrix(errors_nn).T[2]))
#print np.max(np.matrix(errors_nn).T[2])

78.3904743261
1613.2


IndexError: index 2 is out of bounds for axis 0 with size 2

# benchmark:

In [44]:
from utils.data import load_data
from utils.pass_quality import pass_quality

In [ ]:
errors = []

for iteration in range(30):
    print iteration
    data = load_data()
    y_lab, y = data[['pass']].values, data['y'].values
    _, y = y[:int(len(y) * 0.75)], y[int(len(y) * 0.75):]
    _, y_lab = y_lab[:int(len(y_lab) * 0.75)], y_lab[int(len(y_lab) * 0.75):]

    #xgb = XGBClassifier()
    #lr = LogisticRegression()
    #nn = nn_model(X_train)

    #xgb.fit(X_train, y_train)
    #lr.fit(X_train, y_train)
    #nn.fit(X_train, y_train, nb_epoch=4, verbose=False)

    #y_pred_xgb = xgb.predict(X_test)
    #y_pred_lr = lr.predict(X_test)
    #y_pred_nn = (nn.predict(X_test) > 0.5).astype(int).flatten()
    errors.append(pass_quality(y_lab.flatten(), y))

In [54]:
print np.mean(np.matrix(errors).T[0])

print np.mean(np.matrix(errors).T[1])
#print np.max(np.matrix(errors_nn).T[1])

print np.mean(np.matrix(errors).T[2])
#print np.max(np.matrix(errors_nn).T[2])

91.3886916966
1684.2962963
158.703703704


# RNN model

In [22]:
1790.8 / (215.3 + 1790.8)

0.8926773341309008

In [38]:
from utils.utils import morphological_filter
from scipy.ndimage.morphology import (binary_fill_holes, 
                                      binary_closing, 
                                      binary_dilation,
                                      binary_erosion,
                                      binary_opening)

In [35]:
morphological_filter(y_test, 3)[:10]

array([1, 1, 0, 0, 0, 0, 1, 1, 0, 0])

In [37]:
binary_fill_holes(y_test)[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [34]:
y_test[:10]

array([1, 0, 1, 0, 0, 0, 1, 1, 0, 0])

In [60]:
accuracy_score(binary_closing(binary_closing(binary_closing(y_test).astype(int))), morphological_filter(y_test, 3))

0.73518509365887408

In [41]:
from sklearn.metrics import accuracy_score

In [74]:
from IPython.display import SVG
from keras.utils.visualize_util import plot
from keras.layers import SimpleRNN, Dense, Dropout, LSTM
from keras.models import Sequential

from utils.data import load_data, train_test_split
from utils.cv import cv_build_model, cv_fit_model, cv_threshold
from utils.utils import morphological_filter
from utils.pass_quality import approx_pq, pass_quality

import pickle
#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'), filename='model.svg')

In [78]:
def cv_build_model(X_train):#, metrics=[pq_theano]):
    """
    A function for building Recurrent Neural Network model with
    predefined parameters (X_train used because of input_shape
    parameter estimation)

    :param X_train: training dataset
    :return: model: keras neural netwok model
    """
    model = Sequential()
    model.add(LSTM(input_dim=X_train.shape[2], output_dim=8, return_sequences=False))
    model.add(Dropout(0.25))
    model.add(Dense(16, activation='tanh'))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation='tanh'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='tanh'))

    # or with regularized_mse loss
    model.compile(optimizer='rmsprop',
                  loss='MSE',
                #  metrics=metrics
                  )

    return model

In [79]:
used_cols = [['cor', 'shield', 'loop', 'y'],
             ['cor', 'shield', 'y']]


conc_data = load_data()
X_train, X_test, y_train, y_test = train_test_split(conc_data[used_cols[0]], 
                                                            test_size=0.25,
                                                            n_prev=45)

model = cv_build_model(X_train)

model = cv_fit_model(model, X_train, y_train, nb_epoch=80, weights=False)        
        
        #model = cv_fit_model(model, X_train, y_train, nb_epoch=80, weights=False) #DEBUG

thres, sm = cv_threshold(X_train, y_train, model, [0, 1, 2, 3])
        
        #thres, sm = cv_threshold(X_train, y_train, model, [0])#, 1, 2, 3]) # DEBUG
        
y_train_pred = morphological_filter((model.predict(X_train) > thres).astype(int).flatten(), sm)

y_test_pred = morphological_filter((model.predict(X_test) > thres).astype(int).flatten(), sm)

pq = approx_pq(y_test.astype(int), y_test_pred)
pq_train = approx_pq(y_train.astype(int), y_train_pred)
        
pq_Ivan = pass_quality(y_test.astype(int), y_test_pred)

Epoch 79/80
Epoch 1/1
64269/64269 [==============================] - 3s - loss: 0.0933     


In [81]:
print pq[:-1]
print pq_train[:-1]

(0.83662650602488375, 1736.0000000099999, 339.0000000000001)
(0.82645161290350577, 5124.0000000099999, 1076.0)


In [19]:
plot(model, to_file='model.png', show_shapes=True)

# FINAL

In [134]:
from utils.pass_quality import approx_pq, pass_quality

In [175]:
result = pickle.load(open('results/result_cor+shield+loop', 'r'))
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   94.81523834  1764.5           96.5       ]
[  0.9390924   17.62526596  17.47140521]


In [176]:
result = pickle.load(open('results/result_cor+shield', 'r'))
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   94.96903929  1770.8           93.9       ]
[  0.68240379  10.6282642   13.45696846]


In [177]:
result = pickle.load(open('results/result_cor+loop', 'r'))
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   90.95034787  1713.4          170.6       ]
[  1.13912651  16.71645896  22.40178564]


In [178]:
result = pickle.load(open('results/result_shield+loop', 'r'))
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   93.00713864  1738.7          130.8       ]
[  0.8875012   11.92518344  17.18022119]


In [179]:
result = pickle.load(open('results/result_shield', 'r'))
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   89.3257022  1679.5         200.8      ]
[  1.2639559   22.60641502  24.57152824]


In [180]:
result = pickle.load(open('results/result_loop', 'r'))
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   86.37189729  1621.6          255.9       ]
[  0.99158878  18.33684815  19.07590103]


In [181]:
result = pickle.load(open('results/result_cor', 'r'))
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   91.77026954  1714.1          153.9       ]
[  1.8907605   31.28402148  36.33854703]


In [11]:
result = pickle.load(open('results/result_cor+shield', 'r'))
result30 = pickle.load(open('results/result30_cor+shield', 'r'))
result = np.concatenate([result, result30])
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   95.07425072  1769.85          91.75      ]
[  0.63585293  12.08211488  12.28973149]


In [12]:
result = pickle.load(open('results/result_cor+shield+loop', 'r'))
result30 = pickle.load(open('results/result30_cor+shield+loop', 'r'))
result = np.concatenate([result, result30])
print np.mean([result[i]['pass_quality'] for i in range(len(result))], axis=0)
print np.std([result[i]['pass_quality'] for i in range(len(result))], axis=0)

[   94.99541766  1767.6           93.15      ]
[  0.86582434  18.55909481  16.2412284 ]
